In [1]:
import pandas as pd
import os

RAW_DATA_PATH = '../data/raw'

file_names = [
    'olist_customers_dataset.csv',
    'olist_geolocation_dataset.csv',
    'olist_order_items_dataset.csv',
    'olist_order_payments_dataset.csv',
    'olist_order_reviews_dataset.csv',
    'olist_orders_dataset.csv',
    'olist_products_dataset.csv',
    'olist_sellers_dataset.csv',
    'product_category_name_translation.csv'
]

## Verificação de Integridade

In [2]:
print("--- INICIANDO VERIFICAÇÃO DE INTEGRIDADE DOS DADOS ---")

# --- Loop de Verificação ---
for file in file_names:
    file_path = os.path.join(RAW_DATA_PATH, file)
    
    print(f"\n=======================================================")
    print(f"Arquivo: {file}")
    print(f"=======================================================")
    
    try:
        df = pd.read_csv(file_path)
        
        print(f"Dimensões: {df.shape[0]} linhas e {df.shape[1]} colunas.")
        
        print("\nTipos de Dados e Nulos (Resumo):")
        # buffer para capturar a saída do .info()
        import io
        buffer = io.StringIO()
        df.info(buf=buffer)
        print(buffer.getvalue())
        
        # Contagem Detalhada de Nulos (em porcentagem)
        missing_values = df.isnull().sum()
        missing_percentage = (missing_values / len(df)) * 100
        missing_df = pd.DataFrame({'count': missing_values, 'percentage': missing_percentage})
        missing_df = missing_df[missing_df['count'] > 0].sort_values(by='percentage', ascending=False)
        
        if not missing_df.empty:
            print("\nPercentual de Valores Ausentes por Coluna:")
            print(missing_df)
        else:
            print("\nNenhum valor ausente encontrado.")

        # Linhas Duplicadas
        num_duplicates = df.duplicated().sum()
        if num_duplicates > 0:
            print(f"\nAlerta: Encontradas {num_duplicates} linhas duplicadas.")
        else:
            print("\nNenhuma linha duplicada encontrada.")
            
        # Primeiras Linhas
        print("\nPrimeiras 3 Linhas do Dataset:")
        print(df.head(3))
        
    except FileNotFoundError:
        print(f"Erro: O arquivo {file} não foi encontrado no caminho especificado.")
    except Exception as e:
        print(f"Ocorreu um erro ao processar o arquivo {file}: {e}")

print("\n--- VERIFICAÇÃO DE INTEGRIDADE CONCLUÍDA ---")

--- INICIANDO VERIFICAÇÃO DE INTEGRIDADE DOS DADOS ---

Arquivo: olist_customers_dataset.csv
Dimensões: 99441 linhas e 5 colunas.

Tipos de Dados e Nulos (Resumo):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


Nenhum valor ausente encontrado.

Nenhuma linha duplicada encontrada.

Primeiras 3 Linhas do Dataset:
                        customer_id                customer_unique_id  \
0  06b8999e2fba1a1fbc88172c00ba8bc7  861eff4711a542e4b93843c6dd7febb0   
1  18955e83d337fd6b2def6b18a428ac77  290c77bc529b7

### Análise dos resultados da verificação
#### Qualidade Geral
A maioria dos datasets está sem valores nulos ou duplicados. Podemos confiar na integridade das chaves primárias e estrangeiras.
#### Pontos de Atenção
##### Arquivo `olist_orders_dataset.csv` (Valores Nulos em Datas)

Encontramos valores nulos nas colunas de data que rastreiam o progresso da entrega:
- `order_delivered_customer_date` (2.98% nulos)
- `order_delivered_carrier_date` (1.79% nulos)
- `order_approved_at` (0.16% nulos)

Após análise na descrição do dataset, chega-se a conclusão de que estes nulos correspondem ao estado de **pedido não entregue** (ex: foi cancelado ou estava em trânsito no fim da coleta).

###### Ações
Para a criação da variável alvo `atrasado`, precisamos dos pedidos que foram efetivamente entregues. Portanto, na fase de preparação de dados, teremos que filtrar o dataset, mantendo apenas as linhas onde `order_delivered_customer_date` não é nulo. Os pedidos cancelados ou em trânsito não são úteis para treinar um modelo que prevê atraso na entrega.

##### Arquivo `olist_order_reviews_dataset.csv` (Valores Nulos em Comentários)
A maioria dos clientes apenas dá uma nota (review_score) sem se dar ao trabalho de escrever um comentário: `review_comment_title` (88% nulos) e `review_comment_message` (58% nulos).

###### Ações
O escopo do projeto não abrange PLN (Processamento de Linguagem Natural) portanto podemos ignorar ou remover as colunas.

##### Arquivo `olist_products_dataset.csv` (Valores Nulos)
